In [ ]:
dbutils.fs.mount(source='wasbs://landing@assignment11storage.blob.core.windows.net',mount_point='/mnt/assignment11/landing',extra_configs={'fs.azure.account.key.assignment11storage.blob.core.windows.net':'QP11eWllyaiGS+Gh/+9E7504MVEEq2zMkl7w/gk+STE2UAwTf877eljhMot0cGSP1dp+I+tozIki+AStNJk7uA=='})

In [ ]:
dbutils.fs.mount(source='wasbs://staging@assignment11storage.blob.core.windows.net',mount_point='/mnt/assignment11/staging',extra_configs={'fs.azure.account.key.assignment11storage.blob.core.windows.net':'QP11eWllyaiGS+Gh/+9E7504MVEEq2zMkl7w/gk+STE2UAwTf877eljhMot0cGSP1dp+I+tozIki+AStNJk7uA=='})

In [ ]:
dbutils.fs.mount(source='wasbs://reporting@assignment11storage.blob.core.windows.net',mount_point='/mnt/assignment11/reporting',extra_configs={'fs.azure.account.key.assignment11storage.blob.core.windows.net':'QP11eWllyaiGS+Gh/+9E7504MVEEq2zMkl7w/gk+STE2UAwTf877eljhMot0cGSP1dp+I+tozIki+AStNJk7uA=='})

In [ ]:
dbutils.fs.ls('/mnt/assignment11/landing')

[FileInfo(path='dbfs:/mnt/assignment11/landing/customers.csv', name='customers.csv', size=953719, modificationTime=1698648913000),
 FileInfo(path='dbfs:/mnt/assignment11/landing/orders.csv', name='orders.csv', size=2999896, modificationTime=1698648915000)]

In [ ]:
orders_schema = 'order_id string, order_date date, order_customer_id string, order_status string'

In [ ]:
orders = spark.read.format('csv').schema(orders_schema).load('dbfs:/mnt/assignment11/landing/orders.csv',header = True)

In [ ]:
orders.show()

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|2013-07-25|            11599|         CLOSED|
|       2|2013-07-25|              256|PENDING_PAYMENT|
|       3|2013-07-25|            12111|       COMPLETE|
|       4|2013-07-25|             8827|         CLOSED|
|       5|2013-07-25|            11318|       COMPLETE|
|       6|2013-07-25|             7130|       COMPLETE|
|       7|2013-07-25|             4530|       COMPLETE|
|       8|2013-07-25|             2911|     PROCESSING|
|       9|2013-07-25|             5657|PENDING_PAYMENT|
|      10|2013-07-25|             5648|PENDING_PAYMENT|
|      11|2013-07-25|              918| PAYMENT_REVIEW|
|      12|2013-07-25|             1837|         CLOSED|
|      13|2013-07-25|             9149|PENDING_PAYMENT|
|      14|2013-07-25|             9842|     PROCESSING|
|      15|2013-07-25|             2568|       CO

In [ ]:
customers_schema = 'customer_id string, \
customer_fname string, \
customer_lname string, \
customer_email string, \
customer_password string, \
customer_street string, \
customer_city string, \
customer_state string, \
customer_zipcode string' 

In [ ]:
customers = spark.read.format('csv').schema(customers_schema).load('dbfs:/mnt/assignment11/landing/customers.csv',header = True)

In [ ]:
display(customers)

In [ ]:
from pyspark.sql.functions import year, month

In [ ]:
orders = orders.withColumn('order_year',year('order_date'))

In [ ]:
orders.show()

+--------+----------+-----------------+---------------+----------+
|order_id|order_date|order_customer_id|   order_status|order_year|
+--------+----------+-----------------+---------------+----------+
|       1|2013-07-25|            11599|         CLOSED|      2013|
|       2|2013-07-25|              256|PENDING_PAYMENT|      2013|
|       3|2013-07-25|            12111|       COMPLETE|      2013|
|       4|2013-07-25|             8827|         CLOSED|      2013|
|       5|2013-07-25|            11318|       COMPLETE|      2013|
|       6|2013-07-25|             7130|       COMPLETE|      2013|
|       7|2013-07-25|             4530|       COMPLETE|      2013|
|       8|2013-07-25|             2911|     PROCESSING|      2013|
|       9|2013-07-25|             5657|PENDING_PAYMENT|      2013|
|      10|2013-07-25|             5648|PENDING_PAYMENT|      2013|
|      11|2013-07-25|              918| PAYMENT_REVIEW|      2013|
|      12|2013-07-25|             1837|         CLOSED|      2

In [ ]:
orders = orders.withColumn('order_month',month('order_date'))

In [ ]:
orders.show()

+--------+----------+-----------------+---------------+----------+-----------+
|order_id|order_date|order_customer_id|   order_status|order_year|order_month|
+--------+----------+-----------------+---------------+----------+-----------+
|       1|2013-07-25|            11599|         CLOSED|      2013|          7|
|       2|2013-07-25|              256|PENDING_PAYMENT|      2013|          7|
|       3|2013-07-25|            12111|       COMPLETE|      2013|          7|
|       4|2013-07-25|             8827|         CLOSED|      2013|          7|
|       5|2013-07-25|            11318|       COMPLETE|      2013|          7|
|       6|2013-07-25|             7130|       COMPLETE|      2013|          7|
|       7|2013-07-25|             4530|       COMPLETE|      2013|          7|
|       8|2013-07-25|             2911|     PROCESSING|      2013|          7|
|       9|2013-07-25|             5657|PENDING_PAYMENT|      2013|          7|
|      10|2013-07-25|             5648|PENDING_PAYME

In [ ]:
orders.write.format('parquet').mode('overwrite').partitionBy('order_year','order_status').save('/mnt/assignment11/staging/order')

In [ ]:
customers.write.format('parquet').mode('overwrite').partitionBy('customer_state').save('/mnt/assignment11/staging/customer')

In [ ]:
orders_stage = spark.read.format('parquet').option('header','true').load('dbfs:/mnt/assignment11/staging/order')

In [ ]:
customers_stage = spark.read.format('parquet').option('header','true').load('dbfs:/mnt/assignment11/staging/customer')

In [ ]:
orders_stage.show()

+--------+----------+-----------------+-----------+----------+------------+
|order_id|order_date|order_customer_id|order_month|order_year|order_status|
+--------+----------+-----------------+-----------+----------+------------+
|   68883|2014-07-23|             5533|          7|      2014|    COMPLETE|
|   68880|2014-07-13|             1117|          7|      2014|    COMPLETE|
|   68879|2014-07-09|              778|          7|      2014|    COMPLETE|
|   68878|2014-07-08|             6753|          7|      2014|    COMPLETE|
|   68876|2014-07-06|             4124|          7|      2014|    COMPLETE|
|   68874|2014-07-03|             1601|          7|      2014|    COMPLETE|
|   68872|2014-06-29|             3354|          6|      2014|    COMPLETE|
|   68870|2014-06-26|             3343|          6|      2014|    COMPLETE|
|   68859|2014-06-11|             1428|          6|      2014|    COMPLETE|
|   68858|2014-06-06|            10744|          6|      2014|    COMPLETE|
|   68857|20

In [ ]:
customers_stage.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_zipcode|customer_state|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|      12434|          Mary|         Mills|     XXXXXXXXX|        XXXXXXXXX|9720 Colonial Parade|       Caguas|           00725|            PR|
|      12432|        Angela|         Smith|     XXXXXXXXX|        XXXXXXXXX|1525 Jagged Barn ...|       Caguas|           00725|            PR|
|      12430|        Hannah|         Brown|     XXXXXXXXX|        XXXXXXXXX|  8316 Pleasant Bend|       Caguas|           00725|            PR|
|      12428|       Jeffrey|        Travis|     XXXXXXXXX|        XXXXXXXXX|1552 Burning Dale...|       Caguas|           00725|        

In [ ]:
orders_stage.count()

68881

In [ ]:
customers_stage.count()

12434

In [ ]:
from pyspark.sql.functions import broadcast,expr

In [ ]:
joined_df = orders_stage.join(broadcast(customers_stage),expr('order_customer_id=customer_id'))

In [ ]:
reporting_df = joined_df.select("order_id","order_date","order_customer_id", "order_month","order_year","order_status","customer_fname","customer_lname","customer_city","customer_zipcode","customer_state")

In [ ]:
reporting_df.show()

+--------+----------+-----------------+-----------+----------+------------+--------------+--------------+-------------+----------------+--------------+
|order_id|order_date|order_customer_id|order_month|order_year|order_status|customer_fname|customer_lname|customer_city|customer_zipcode|customer_state|
+--------+----------+-----------------+-----------+----------+------------+--------------+--------------+-------------+----------------+--------------+
|   68883|2014-07-23|             5533|          7|      2014|    COMPLETE|          Mary|      Harrison|       Caguas|           00725|            PR|
|   68880|2014-07-13|             1117|          7|      2014|    COMPLETE|        Arthur|        Maddox|       Caguas|           00725|            PR|
|   68879|2014-07-09|              778|          7|      2014|    COMPLETE|          John|       Buckley|       Caguas|           00725|            PR|
|   68878|2014-07-08|             6753|          7|      2014|    COMPLETE|        Judit

In [ ]:
reporting_df.write.format('parquet').mode('overwrite').partitionBy('order_year','customer_state','customer_city','order_status').save('/mnt/assignment11/reporting')

In [ ]:
reporting_df.write.format('parquet')\
  .mode('overwrite')\
  .partitionBy('order_year', 'customer_state', 'customer_city', 'order_status')\
  .save('/mnt/assignment11/reporting')


In [ ]:
dbutils.fs.ls('dbfs:/mnt/assignment11/reporting')

[FileInfo(path='dbfs:/mnt/assignment11/reporting/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1698696978000),
 FileInfo(path='dbfs:/mnt/assignment11/reporting/order_year=2013/', name='order_year=2013/', size=0, modificationTime=1698656991000),
 FileInfo(path='dbfs:/mnt/assignment11/reporting/order_year=2014/', name='order_year=2014/', size=0, modificationTime=1698656991000),
 FileInfo(path='dbfs:/mnt/assignment11/reporting/report/', name='report/', size=0, modificationTime=1698657535000)]

In [ ]:
reporting = spark.read.format('parquet').option('header','true').load('dbfs:/mnt/assignment11/reporting')

In [ ]:
new_reprting = reporting.select(expr("order_customer_id").alias("customer_id"), expr("concat(customer_fname,' ',customer_lname)").alias("customer_name"), "customer_city", "order_date", "order_status", "customer_state", "order_year", "order_month")

In [ ]:
new_reporting.createOrReplaceTempView('reporting_view')

In [ ]:
spark.sql("select * from reporting_view limit 10").show()

+-----------+-----------------+-------------+----------+------------+--------------+----------+-----------+
|customer_id|    customer_name|customer_city|order_date|order_status|customer_state|order_year|order_month|
+-----------+-----------------+-------------+----------+------------+--------------+----------+-----------+
|       6735|   Barbara Martin|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       1044|       Linda Pope|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       6405|    Mary Anderson|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       3099|Brittany Copeland|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       9382|        Mary Rowe|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       9288|       Mary Mason|       Caguas|2014-01-01|    COMPLETE|            PR|      2014|          1|
|       3219|      Mary Fren

In [ ]:
spark.sql("cache table reporting_view")

DataFrame[]

In [ ]:
%sql
select distinct customer_name from reporting_view where customer_state='TX' and order_status='COMPLETE' and order_year='2014';

customer_name
Elizabeth Ball
Mary Patel
Mary Baird
Sandra Smith
Christina Walsh
Arthur Fisher
Mary Patterson
Raymond Richards
Christopher Smith
Janice Novak


In [ ]:
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [ ]:
states_df = spark.sql("select distinct customer_state from reporting_view")

In [ ]:
states_df.show()

+--------------+
|customer_state|
+--------------+
|            NV|
|            CA|
|            IL|
|            PA|
|            NY|
|            TX|
|            FL|
|            PR|
|            AZ|
|            VA|
|            MI|
|            OH|
|            CO|
|            MN|
|            DC|
|            MD|
|            TN|
|            HI|
|            LA|
|            OR|
+--------------+
only showing top 20 rows



In [ ]:
states_list = df.rdd.flatMap(lambda x: x).collect() 

In [ ]:
dbutils.widgets.dropdown(name='state_input',defaultValue='TX',choices = states_list, label="State:")

In [ ]:
dbutils.widgets.dropdown(name='status_input',defaultValue='COMPLETE',choices=['COMPLETE','PENDING_PAYMENT','CLOSED','PROCESSING'],label="Status:")

In [ ]:
dbutils.widgets.text(name='year_input',defaultValue='2014',label="Year:")

In [ ]:
state = dbutils.widgets.get('state_input')

In [ ]:
status = dbutils.widgets.get('status_input')

In [ ]:
year = dbutils.widgets.get('year_input')

In [ ]:
spark.sql(f"Select distinct customer_name from reporting_view where customer_state='{state}' and order_status='{status}' and order_year='{year}'").show()

+-------------------+
|      customer_name|
+-------------------+
|         Mary Patel|
|         Mary Baird|
|       Sandra Smith|
|      Arthur Fisher|
|       Janice Novak|
|         Juan Smith|
|         Lori Smith|
|         Mary Lopez|
|          Roy Smith|
|        Helen Gomez|
|   Richard Richmond|
|         Mary Smith|
|        Keith Smith|
|       Mary Jimenez|
|         Peter Bray|
|         Helen Ward|
|Christine Rodriguez|
|        Mary Garner|
|      William Smith|
|      Stephen Smith|
+-------------------+
only showing top 20 rows



In [ ]:
dbutils.widgets.dropdown(name='month',defaultValue='1',choices=[str(i) for i in range(1,13)],label='Month:')

In [ ]:
month=dbutils.widgets.get('month')

In [ ]:
spark.sql(f"select count(*) as total_customers from reporting_view where order_month='{month}' and order_status='{status}' and order_year='{year}' and customer_state='{state}'").show()

+---------------+
|total_customers|
+---------------+
|            118|
+---------------+

